In [18]:
import pandas as pd
import numpy as np
from scipy.stats import kurtosis, skew
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from scipy.stats import mode
import matplotlib
import pickle

In [19]:
matplotlib.use("TkAgg")
from sklearn.model_selection import train_test_split

In [20]:
# Window Length.
n = 100
STRIDE = 50

In [21]:
def getFeatures(data, fs):
    f1 = np.mean(data, axis=0)
    C = np.cov(data.T)
    f2 = np.concatenate((C[0, 0:3], C[1, 1:3], C[2, 2:3]))
    f3 = np.array([skew(data.iloc[:, 0]), skew(data.iloc[:, 1]), skew(data.iloc[:, 2])])
    f4 = np.array([kurtosis(data.iloc[:, 0]), kurtosis(data.iloc[:, 1]), kurtosis(data.iloc[:, 2])])
    f5 = np.zeros(3)
    f6 = np.zeros(3)
    for i in range(0,3):
        g = abs(np.fft.fft(data.iloc[:,i]))
        g = g[0:round(len(g)/2)]
        g[0] = 0
        max_i = np.argmax(g)
        f5[i] = g[max_i]
        f6[i] = fs * max_i
    return np.concatenate((f1, f2, f3, f4, f5, f6))

In [22]:
def generate_features(df, stride=STRIDE):
    df_out = pd.DataFrame(columns=[i for i in range(0, 42)])
    fs = 1 / n
    i = 0
    for i in range(0, df.shape[0] - n, stride):
        features_accel = getFeatures(df.iloc[i:i+n, 0:3], fs)
        features_gyro = getFeatures(df.iloc[i:i+n, 3:6], fs)
        features = np.concatenate((features_accel, features_gyro))
        df_out.loc[i] = features
        i += 1
        print(i)
    return df_out

In [31]:
# choose via MODE ## TBD ##
def get_detection_window(df, stride=STRIDE):
    df_out = pd.DataFrame(columns=[0])
    i = 0
    for i in range(0, df.shape[0] - n, stride):
        df_slice = df.iloc[i:i+n,:]
        df_out.loc[i] = df_slice.mode().iloc[0,0]
#         df_out.loc[i] = df.iloc[i+n-1]
        i += 1
    return df_out

In [32]:
def extrapolate_predictions(predictions, size):
    extrapolated = np.fromiter((val for val in predictions for _ in range(0, STRIDE)), float)
    return extrapolated[0:size]

In [33]:
def read_test_data():
    df_data_arm = pd.read_csv('SampleData/armIMU.txt', delim_whitespace=True, header=None)
    _n = df_data_arm.shape[0]
    arm_features = generate_features(df_data_arm, stride=1)
    df_data_wrist = pd.read_csv('SampleData/wristIMU.txt', delim_whitespace=True, header=None)
    wrist_features = generate_features(df_data_wrist, stride=1)
    df_data_detection = pd.read_csv('SampleData/detection.txt', header=None)
    df_data_detection = get_detection_window(df_data_detection, stride=1)
    zero_padding = pd.DataFrame(np.zeros(n))
    return _n, pd.concat([arm_features, wrist_features], axis=1, sort=False, ignore_index=True), zero_padding.append(df_data_detection)

In [34]:
def validate_data(x_val,y_val):
    x_train, x_test, y_train, y_test = train_test_split( x_val, y_val, test_size=0.2, random_state=42)
    clf = SVC(gamma='auto')
    clf.fit(x_train, y_train)
    # Save the model.
    filename = 'finalized_model.sav'
    pickle.dump(clf, open(filename, 'wb'))
    
    predictions = clf.predict(x_test)
    score = accuracy_score(y_test, predictions)
    print('The Validation accuracy score of the model {}'.format(score))

In [35]:
def read_data():
    # sessions = ['Session01', 'Session05', 'Session06', 'Session07', 'Session12', 'Session13']
    sessions = ['Session01']
    # setting empty dataframes for training
    x_train = pd.DataFrame(columns=[i for i in range(0, 84)])
    y_train = pd.DataFrame(columns=[0])

    for session in sessions:
        df_data_arm = pd.read_csv('TrainingData/{}/armIMU.txt'.format(session),  delim_whitespace=True, header=None)
        arm_features = generate_features(df_data_arm)
        df_data_wrist = pd.read_csv('TrainingData/{}/wristIMU.txt'.format(session), delim_whitespace=True, header=None)
        wrist_features = generate_features(df_data_wrist)
        features = pd.concat([arm_features, wrist_features], axis=1, sort=False, ignore_index=True)
        df_data_detection = pd.read_csv('TrainingData/{}/detection.txt'.format(session), header=None)
        x_train = x_train.append(other=features, ignore_index=True)
        y_train = y_train.append(other=get_detection_window(df_data_detection), ignore_index=True)

    validate_data(x_train, y_train)

    # Testing
    size, x_test, y_test = read_test_data()
    
    filename = 'finalized_model.sav'
    clf = pickle.load(open(filename, 'rb'))
    
    predictions = clf.predict(x_test)
    predictions = extrapolate_predictions(predictions, size)
    score = accuracy_score(y_test, predictions)
    print('The accuracy score of the model {}'.format(score))


In [ ]:
read_data()

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277
